<a href="https://colab.research.google.com/github/GenaroHacker/creating_chord_collection/blob/main/detect_chord_coords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Set Up
%%capture
!git clone https://github.com/GenaroHacker/creating_chord_collection.git

In [2]:
# @title Extract chords with coords
import zipfile

# Define the path to the zip file and the extraction directory
zip_file_path = '/content/creating_chord_collection/coord_images.zip'
extraction_directory = '/content/chords_with_coords/'

# Using zipfile to extract the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_directory)

# This code will extract the contents of 'coord_images.zip' into the specified directory.


In [3]:
# @title Draw Coordenates
path_to_original_chord = "/content/chords_with_coords/Ddim.png" # @param {type:"string"}
import cv2
import numpy as np
from PIL import Image

def process_image(image_path, threshold, pixel_threshold):
    def convert_to_black_and_white(image, threshold):
        grey_image = image.convert("L")
        image_array = np.array(grey_image)
        normalized_image_array = image_array / 255.0
        image_array[normalized_image_array > threshold] = 255
        image_array[normalized_image_array <= threshold] = 0
        return Image.fromarray(image_array)

    def detect_rectangle_edges_full_axis(image, pixel_threshold):
        img_gray = np.array(image.convert('L'))
        img = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2BGR)
        _, binary = cv2.threshold(img_gray, 128, 255, cv2.THRESH_BINARY_INV)

        top_edge, bottom_edge = 0, binary.shape[0] - 1
        left_edge, right_edge = 0, binary.shape[1] - 1

        for row in range(binary.shape[0]):
            if np.sum(binary[row, :] == 255) >= pixel_threshold:
                top_edge = row
                break

        for row in range(binary.shape[0] - 1, -1, -1):
            if np.sum(binary[row, :] == 255) >= pixel_threshold:
                bottom_edge = row
                break

        for col in range(binary.shape[1]):
            if np.sum(binary[:, col] == 255) >= pixel_threshold:
                left_edge = col
                break

        for col in range(binary.shape[1] - 1, -1, -1):
            if np.sum(binary[:, col] == 255) >= pixel_threshold:
                right_edge = col
                break

        cv2.line(img, (0, top_edge), (img.shape[1], top_edge), (0, 0, 255), 1)
        cv2.line(img, (0, bottom_edge), (img.shape[1], bottom_edge), (0, 0, 255), 1)
        cv2.line(img, (left_edge, 0), (left_edge, img.shape[0]), (255, 0, 0), 1)
        cv2.line(img, (right_edge, 0), (right_edge, img.shape[0]), (255, 0, 0), 1)

        return img

    def process_final_image(image):
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        blue_lower, blue_upper = np.array([0, 0, 100], dtype="uint8"), np.array([50, 50, 255], dtype="uint8")
        red_lower, red_upper = np.array([100, 0, 0], dtype="uint8"), np.array([255, 50, 50], dtype="uint8")

        blue_mask = cv2.inRange(image_rgb, blue_lower, blue_upper)
        red_mask = cv2.inRange(image_rgb, red_lower, red_upper)

        blue_contours, _ = cv2.findContours(blue_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        red_contours, _ = cv2.findContours(red_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        blue_lines_x = sorted([cv2.boundingRect(contour)[0] for contour in blue_contours])
        red_lines_y = sorted(set([cv2.boundingRect(contour)[1] for contour in red_contours]))

        vertical_distance = np.abs(blue_lines_x[0] - blue_lines_x[1])
        horizontal_distance = np.abs(red_lines_y[0] - red_lines_y[1])
        N = horizontal_distance // 4

        # Draw intermediate blue lines
        blue_intermediate_positions = np.linspace(blue_lines_x[0], blue_lines_x[1], 6)
        for x in blue_intermediate_positions:
            cv2.line(image, (int(x), 0), (int(x), image.shape[0]), (255, 0, 0), 1)

        # Draw green horizontal lines
        first_green_line_position = red_lines_y[0] - (N // 2)
        green_line_positions = [first_green_line_position + 10]  # Move first green line 100 pixels lower
        green_line_positions += [first_green_line_position + (i + 1) * N for i in range(4)]  # Other green lines remain the same

        for y in green_line_positions:
            cv2.line(image, (0, int(y)), (image.shape[1], int(y)), (0, 255, 0), 1)

        return image

    # Start of the main function process
    original_image = Image.open(image_path)
    bw_image = convert_to_black_and_white(original_image, threshold)
    edge_detected_image = detect_rectangle_edges_full_axis(bw_image, pixel_threshold)
    final_image = process_final_image(edge_detected_image)

    final_output_path = 'final_processed_image.png'
    cv2.imwrite(final_output_path, final_image)
    return final_output_path

# Example usage
image_path = path_to_original_chord
threshold = 0.5
pixel_threshold = 200
output_path = process_image(image_path, threshold, pixel_threshold)
print(output_path)


final_processed_image.png


In [4]:
# @title Find Intersections
from typing import Dict, Tuple

def find_image_intersections_final(image: np.ndarray) -> Dict[Tuple[int, int], Tuple[int, int]]:
    # Initialize dictionaries to hold the line positions and intersections
    line_positions = {'green': {}, 'blue': {}}
    intersections = {}

    # Define the color thresholds for green and blue lines in BGR format
    green_color = (0, 255, 0)
    blue_color = (255, 0, 0)

    # Extract the green and blue lines
    for y in range(image.shape[0]):
        for x in range(image.shape[1]):
            if tuple(image[y, x]) == green_color:
                line_positions['green'][y] = True
            elif tuple(image[y, x]) == blue_color:
                line_positions['blue'][x] = True

    # Sort the line positions
    sorted_green_lines = sorted(line_positions['green'].keys())
    sorted_blue_lines = sorted(line_positions['blue'].keys())

    # Determine intersections
    for y_index, y in enumerate(sorted_green_lines):
        for x_index, x in enumerate(sorted_blue_lines):
            intersections[(y, x)] = (y_index, x_index)

    intersections = {(x, y): val for (y, x), val in intersections.items()}
    return intersections

import cv2

# Load the image
image = cv2.imread('/content/final_processed_image.png')

# Now you can pass this image to your function
final_intersections = find_image_intersections_final(image)


In [5]:
# @title Draw red circles and relatie points
from PIL import Image, ImageDraw, ImageFont

def annotate_image_with_coordinates(image_path, coordinates_dict):
    # Load the image
    image = Image.open(image_path)

    # Create a drawing context
    draw = ImageDraw.Draw(image)

    # Use a default font provided by PIL/Pillow
    font = ImageFont.load_default()

    # Define circle properties
    radius = 10  # Radius of the circle

    # Process each coordinate and annotation
    for position, text in coordinates_dict.items():
        # Convert the tuple to a string with parentheses
        text_str = f"({text[0]}, {text[1]})"

        # Draw the red circle around the coordinate
        circle_bbox = [position[0] - radius, position[1] - radius, position[0] + radius, position[1] + radius]
        draw.ellipse(circle_bbox, outline="red", width=1)

        # Draw the text on the image
        draw.text(position, text_str, font=font, fill="black")

    # Return the annotated image
    return image

# Now, we can use this refactored function with the original parameters.
image_path = '/content/final_processed_image.png'

# Annotate the image
annotated_image = annotate_image_with_coordinates(image_path, final_intersections)

# Save the edited image
edited_image_path_refactored = 'path_to_save_annotated_image.png'
annotated_image.save(edited_image_path_refactored)


In [6]:
# @title Detect upper row figures
import cv2
import numpy as np
from typing import Tuple

# Define the valid colors globally
valid_colors = {
    'red': np.array([0, 0, 255]),
    'green': np.array([0, 255, 0]),
    'blue': np.array([255, 0, 0]),
    'black': np.array([0, 0, 0]),
    'white': np.array([255, 255, 255])
}

def is_valid_color(pixel, valid_colors):
    """ Check if the pixel color is one of the valid colors """
    return any(np.array_equal(pixel, color) for color in valid_colors)

def validate_image_colors(image, valid_colors):
    """ Validate that the image contains only the specified valid colors """
    for row in image:
        for pixel in row:
            if not is_valid_color(pixel, valid_colors):
                return False
    return True

def move_in_direction(x, y, direction, steps=1):
    """ Move the coordinates in the given cardinal direction """
    if direction == 'east':
        return x + steps, y
    elif direction == 'west':
        return x - steps, y
    elif direction == 'north':
        return x, y - steps
    elif direction == 'south':
        return x, y + steps

def count_black_pixels(image, start_x, start_y, direction, distance):
    """ Count the black pixels in a given direction and distance from the start point """
    count = 0
    for step in range(distance):
        x, y = move_in_direction(start_x, start_y, direction, step)
        if is_valid_color(image[y, x], [valid_colors['black']]):
            count += 1
    return count

def detect_figure_a(image, c_center):
    """ Detect Figure A based on the specified rules """
    x, y = move_in_direction(*c_center, 'east')
    x, y = move_in_direction(x, y, 'south', 1)  # Moving south until a red pixel is found
    while not is_valid_color(image[y, x], [valid_colors['red']]):
        x, y = move_in_direction(x, y, 'south', 1)

    x, y = move_in_direction(x, y, 'north')
    x, y = move_in_direction(x, y, 'north')
    c_bottom = move_in_direction(x, y, 'west')

    bottom_counter = count_black_pixels(image, *c_bottom, 'east', 8)
    bottom_counter += count_black_pixels(image, *c_bottom, 'west', 8)

    return 'A' if bottom_counter > 3 else None

def detect_figure_b(image, c_center):
    """ Detect Figure B based on the specified wrap-like path rules """
    directions = ['east', 'south', 'west', 'west', 'north', 'north', 'east', 'east', 'east', 'south', 'south', 'south', 'west', 'west', 'west', 'west', 'north', 'north', 'north', 'north', 'east', 'east', 'east', 'east']
    center_counter = 0
    x, y = c_center

    for direction in directions:
        x, y = move_in_direction(x, y, direction)
        if is_valid_color(image[y, x], [valid_colors['black']]):
            center_counter += 1

    return 'B' if center_counter > 0 else None

def detect_figure(image_path: str, c_center: Tuple[int, int]) -> str:
    """ Detects a figure in an image based on given rules """
    # Load the image
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)

    # Validate the colors in the image
    if not validate_image_colors(img, valid_colors.values()):
        raise ValueError("Image contains invalid colors")

    # Detect Figure A
    figure_a_result = detect_figure_a(img, c_center)
    if figure_a_result:
        return figure_a_result

    # Detect Figure B
    figure_b_result = detect_figure_b(img, c_center)
    if figure_b_result:
        return figure_b_result

    # If neither Figure A nor B is detected, return 'C'
    return 'C'

# Script to iterate over the dictionary and create a list of keys whose value tuple starts with 0


# Create a dictionary where the key is the coordinate and the value is the returned string from detect_figure
coord_to_figure_dict = {coords: detect_figure("/content/final_processed_image.png", coords)
                        for coords in final_intersections
                        if final_intersections[coords][0] == 0}

# Example usage of the new dictionary
for coords, figure in coord_to_figure_dict.items():
    print(f"Coords: {coords}, Figure: {figure}")  # This will print each coordinate and its corresponding figure



Coords: (52, 158), Figure: B
Coords: (97, 158), Figure: B
Coords: (142, 158), Figure: A
Coords: (188, 158), Figure: C
Coords: (233, 158), Figure: A
Coords: (279, 158), Figure: C


In [10]:
# @title Color the image
from PIL import Image, ImageDraw
import os

def paint_squares(coordinates_dict, image_path, transparency):
    # Load the image
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image, 'RGBA')

    # Define the square size
    square_size = 35

    # Iterate through the dictionary and draw squares
    for coordinate, letter in coordinates_dict.items():
        upper_left = (coordinate[0] - square_size // 2, coordinate[1] - square_size // 2)
        lower_right = (coordinate[0] + square_size // 2, coordinate[1] + square_size // 2)

        # Define color based on the letter
        if letter == 'A':
            color = (255, 255, 0, int(255 * transparency))  # Yellow
        elif letter == 'B':
            color = (255, 0, 0, int(255 * transparency))  # Red
        else:
            continue  # Skip if the letter is not 'A' or 'B'

        # Draw the square
        draw.rectangle([upper_left, lower_right], fill=color)

    # Check if the 'colored' directory exists, if not, create it
    colored_dir = '/content/colored/'
    if not os.path.exists(colored_dir):
        os.makedirs(colored_dir)

    # Use the original image name for the processed image
    original_image_name = os.path.basename(image_path)
    processed_image_path = os.path.join(colored_dir, original_image_name)

    # Save the modified image with the new filename
    image.save(processed_image_path)

# Example usage
paint_squares(coord_to_figure_dict, "/content/final_processed_image.png", 0.5)


In [12]:
# @title Process and download all images and image data
import os
import json
import shutil
from glob import glob
import cv2

# Ensure the directory for processed images exists
processed_images_dir = '/content/processed_images/'
if not os.path.exists(processed_images_dir):
    os.makedirs(processed_images_dir)

# Create the dictionary to store results
results_dict = {}

# Iterate over each image in the directory
for image_path in glob('/content/chords_with_coords/*.png'):
    image_name = os.path.basename(image_path)

    # Process each image and save with a unique name
    processed_image_path_temp = process_image(image_path, threshold, pixel_threshold)
    unique_processed_image_path = os.path.join(processed_images_dir, image_name)
    shutil.move(processed_image_path_temp, unique_processed_image_path)

    image = cv2.imread(unique_processed_image_path)
    final_intersections = find_image_intersections_final(image)

    coord_to_figure_dict = {coords: detect_figure(unique_processed_image_path, coords)
                            for coords in final_intersections
                            if final_intersections[coords][0] == 0}

    # Paint squares on the uniquely named image and save it in the 'colored' directory
    paint_squares(coord_to_figure_dict, unique_processed_image_path, 0.5)

    # Store the results in the dictionary
    results_dict[image_name] = {
        'final_intersections': final_intersections,
        'coord_to_figure_dict': coord_to_figure_dict
    }

# Convert tuple keys to string before JSON serialization
results_dict_str_keys = {}
for image_name, data in results_dict.items():
    final_intersections_str = {str(k): v for k, v in data['final_intersections'].items()}
    coord_to_figure_dict_str = {str(k): v for k, v in data['coord_to_figure_dict'].items()}
    results_dict_str_keys[image_name] = {
        'final_intersections': final_intersections_str,
        'coord_to_figure_dict': coord_to_figure_dict_str
    }

# Export the results dictionary as a JSON file
with open('/content/results_dict.json', 'w') as file:
    json.dump(results_dict_str_keys, file)

# Compress the '/content/colored' directory
shutil.make_archive('/content/colored_images', 'zip', '/content/colored')

# Download the JSON file and ZIP file automatically if running in Google Colab
try:
    from google.colab import files
    files.download('/content/results_dict.json')
    files.download('/content/colored_images.zip')
except ImportError:
    print("Automatic download is available only in Google Colab.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>